In [1]:
#settings 
from settings.settings import data_settings 
from settings.settings import model_settings

#imports
import pandas as pd 
import numpy as np 
# import tensorflow as tf
# import keras
# from keras import layers
# import pathlib
# import datetime
# from model.blocks.keras_tcn import TCN

In [2]:
# Loading data and making it ready

train_df = pd.read_csv(data_settings.data_train_path)

test_df = pd.read_csv(data_settings.data_test_path)

test_buy = test_df.pop(test_df.buy_4.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.sell_4.name)

train_buy = train_df.pop(train_df.buy_4.name)
train_nothing = train_df.pop(train_df.do_nothing.name)
train_sell = train_df.pop(train_df.sell_4.name)

train_date_time = train_df.pop(train_df.DateTime.name)
test_date_time = test_df.pop(test_df.DateTime.name)


In [5]:
# Normalizing with z-score https://developers.google.com/machine-learning/data-prep/transform/normalization

# train_df.mean().to_csv(f"{data_settings.base_path}/data/csv/train_mean.csv")
# train_df.std().to_csv(f"{data_settings.base_path}/data/csv/train_std.csv")
# train_df = (train_df - train_df.mean())/train_df.std()
# test_df = (test_df - train_df.mean())/train_df.std()

# Corrilation of buy and sell and do nothing with other rows of dataframes
# pd.options.display.max_rows =1000

# train_df.corr()["buy_4"]
# train_df.corr()["sell_4"]
# train_df.corr()["do_nothing"]

# import matplotlib.pyplot as plt

# for column in df.columns:
    
#     hist = df[column].hist(bins=100)
#     plt.savefig(f"./tmp/{column}_normalized.png")
#     hist.clear()


In [3]:
# y_train = pd.DataFrame([train_buy,train_nothing,train_sell]).transpose()
# y_test = pd.DataFrame([test_buy,test_nothing,test_sell]).transpose()


X_train = (train_df - train_df.mean())/train_df.std()
X_test = (test_df - train_df.mean())/train_df.std()

y_train = []
for buy,nothing,sell in zip(train_buy,train_nothing,train_sell):
    if(buy == 1):
        y_train.append(0)
    elif(nothing == 1):
        y_train.append(1)
    elif(sell == 1):
        y_train.append(2)

y_test = []
for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
    if(buy == 1):
        y_test.append(0)
    elif(nothing == 1):
        y_test.append(1)
    elif(sell == 1):
        y_test.append(2)     
# print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)


In [13]:
from xgboost import XGBRFClassifier
from sklearn.metrics import log_loss , precision_score

# model = XGBRFClassifier(
#     learning_rate = 0.001,
#     n_estimators = 100_000,
#     verbosity = 3,
#     tree_method = "gpu_hist",
#     random_state = 23,  
#     gpu_id = 0,
#     eval_metric = log_loss
# )

size = 1_000
max_depth=12


model = XGBRFClassifier(
                        n_estimators=size,
                        max_depth=max_depth,
                        max_leaves=0,
                        max_bin=256,
                        grow_policy='depthwise',
                        learning_rate=1,
                        verbosity = 3,
                        booster='gbtree',
                        tree_method = "gpu_hist",
                        n_jobs=0,
                        gamma=0,
                        min_child_weight=1,
                        sampling_method='gradient_based',
                        colsample_bytree=1,
                        colsample_bylevel=1,
                        reg_alpha=0,
                        base_score=0.5,
                        random_state=23,
                        missing=np.nan,
                        num_parallel_tree=size,
                        monotone_constraints='()',
                        interaction_constraints='',
                        importance_type=None,
                        gpu_id=0,
                        predictor='auto',
                        enable_categorical=False,
                        # max_cat_to_onehot=4,
                        # eval_metric = log_loss,
                        early_stopping_rounds=None,
                        callbacks=None,
                        max_delta_step=0,
                        num_class = 3,
                        objective='multi:softmax',
                        )

In [14]:
# eval_set = [(X_test, y_test)]

model.fit(
            X_train,
            y_train,
            # eval_set = eval_set,
            verbose = 3
        )


[03:51:25] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_gpu_hist.cu:817: [GPU Hist]: Configure
[03:51:26] ======== Monitor (0): SketchContainer ========
[03:51:26] MakeCuts: 0.001241s, 1 calls @ 1241us

[03:51:26] Prune: 0.001027s, 1 calls @ 1027us

[03:51:26] ScanInput: 0.004099s, 1 calls @ 4099us

[03:51:26] Unique: 0.000316s, 1 calls @ 316us



In [10]:
p_train = model.predict(X_train)
p_test = model.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
import pathlib

acc_train = accuracy_score(p_train,y_train)
acc_test = accuracy_score(p_test,y_test)

rep_train = classification_report(y_train,p_train)
rep_test = classification_report(y_test,p_test)

con_train = confusion_matrix(y_train,p_train)
con_test = confusion_matrix(y_test,p_test)


#check if path is available
path = f'./model/models/XGBOOST/model_{size}_{max_depth}'
pathlib.Path(path).mkdir(parents=True,exist_ok=True)
model.save_model(f"{path}/model.json")

with open(f"{path}/train_report.txt","w") as f:
    f.write(rep_train)

with open(f"{path}/test_report.txt","w") as f:
    f.write(rep_test) 
    
print(rep_train)
print(rep_test)


c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\xgboost\sklearn.py:722: UserWarning: eval_metric is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.', UserWarning)


              precision    recall  f1-score   support

           0       0.98      0.16      0.28     44559
           1       0.71      1.00      0.83    189479
           2       0.98      0.11      0.20     44666

    accuracy                           0.72    278704
   macro avg       0.89      0.42      0.44    278704
weighted avg       0.80      0.72      0.64    278704

              precision    recall  f1-score   support

           0       0.17      0.34      0.23      6886
           1       0.70      0.67      0.68     33141
           2       0.18      0.08      0.11      9157

    accuracy                           0.51     49184
   macro avg       0.35      0.36      0.34     49184
weighted avg       0.53      0.51      0.51     49184





1000 , 10
              precision    recall  f1-score   support

           0       0.98      0.16      0.28     44559
           1       0.71      1.00      0.83    189479
           2       0.98      0.11      0.20     44666

    accuracy                           0.72    278704
   macro avg       0.89      0.42      0.44    278704
weighted avg       0.80      0.72      0.64    278704

              precision    recall  f1-score   support

           0       0.17      0.34      0.23      6886
           1       0.70      0.67      0.68     33141
           2       0.18      0.08      0.11      9157

    accuracy                           0.51     49184
   macro avg       0.35      0.36      0.34     49184
weighted avg       0.53      0.51      0.51     49184

BASE
              precision    recall  f1-score   support

           0       0.89      0.05      0.09     44559
           1       0.69      0.99      0.81    189479
           2       0.89      0.03      0.06     44666

   micro avg       0.69      0.69      0.69    278704
   macro avg       0.82      0.36      0.32    278704
weighted avg       0.75      0.69      0.58    278704
 samples avg       0.68      0.69      0.69    278704

              precision    recall  f1-score   support

           0       0.07      0.01      0.01      6886
           1       0.69      0.91      0.78     33141
           2       0.27      0.00      0.01      9157

   micro avg       0.68      0.62      0.65     49184
   macro avg       0.34      0.31      0.27     49184
weighted avg       0.52      0.62      0.53     49184
 samples avg       0.61      0.62      0.61     49184


In [ ]:
from xgboost import plot_importance ,plot_tree
import matplotlib as mpl

mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (200, 100)

plot_tree(model)


In [ ]:
# Creating generators

# window_size = data_settings.window_size
# batch_size : int = 256
# num_features : int = 97
# num_classes : int = 3

# def x_train_generator():
#     for i in range(window_size,len(train_df)+1):
#         window=train_df[i-window_size:i]
#         yield window.to_numpy()

# def x_test_generator():
#     for i in range(window_size,len(test_df)+1):
#         window=test_df[i-window_size:i]
#         yield window.to_numpy()    
        
# def y_train_generator():
#     for i in range(window_size,train_labels.shape[0]+1):
#         yield train_labels[i-1]

# def y_test_generator():
#     for i in range(window_size,test_labels.shape[0]+1):
#         yield test_labels[i-1]
        
# x_train_ds = tf.data.Dataset.from_generator(x_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)

# x_test_ds = tf.data.Dataset.from_generator(x_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)
# y_train_ds = tf.data.Dataset.from_generator(y_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)
# y_test_ds = tf.data.Dataset.from_generator(y_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)

# train_ds = tf.data.Dataset.zip((x_train_ds,y_train_ds))
# test_ds = tf.data.Dataset.zip((x_test_ds,y_test_ds))

In [ ]:
# def compile_and_fit(model:keras.Model,modelname,train_ds=train_ds,test_ds=test_ds,modelcheckpoint:bool = False,epochs= 10):
#     import os

#     #check if path is available
#     path = f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit'
#     pathlib.Path(path).mkdir(parents=True,exist_ok=True)

#     #tensorboard
#     log_dir =f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit/{datetime.datetime.now().strftime("(%Y-%m-%d)-(%H-%M-%S)")}'
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#     #modelcheckpoint
#     base_path = f"./model/models/{modelname}/"
#     checkpoint_path = base_path + "tmp/cp-{epoch:04d}.ckpt"
#     checkpoint_dir = os.path.dirname(checkpoint_path)

#     model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#         filepath=checkpoint_path,
#         monitor='val_loss',
#         verbose=1,
#         save_weights_only=True,
#         save_best_only=False
#     )

#     model.save_weights(checkpoint_path.format(epoch=0))


#     model.compile(loss='categorical_crossentropy',
#                     optimizer='adam',
#                     metrics=["categorical_accuracy","categorical_crossentropy"],
#                 )

#     if modelcheckpoint:
#         history = model.fit(
#                             train_ds,
#                             validation_data= test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback,model_checkpoint_callback],
#                             use_multiprocessing=True,
#                             )
#     else:
#         history = model.fit(    
#                             train_ds,
#                             validation_data=test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback],
#                             use_multiprocessing=True,
#                            )
    
#     return history

In [ ]:
#TF Model
# input_layer = layers.Input(shape=(window_size,num_features))
# x = TCN(
#     nb_filters=128,
#     # kernel_size=3,
#     # nb_stacks=1,
#     # padding="causal",
#     # activation="swish",
#     use_batch_norm=True,
#     # return_sequences=True
#     )(input_layer)

# # x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )

# input_layer = layers.Input(shape=(window_size,num_features))

# x = layers.Dense(1024,activation="relu")(input_layer)
# x = layers.Dense(512,activation="relu")(x)
# x = layers.Dense(256,activation="relu")(x)
# x = layers.Dense(128,activation="relu")(x)
# x = layers.Dense(64,activation="relu")(x)
# x = layers.Dense(32,activation="relu")(x)
# x = layers.Dense(16,activation="relu")(x)
# x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )
# model.summary()
# compile_and_fit(
#     model=model,
#     modelname="simple_dense_first_try",
#     train_ds=train_ds,
#     test_ds=test_ds,
#     modelcheckpoint=True
# )